In [1]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
from sklearn.neighbors import KDTree
import tsfresh as ts 
import matplotlib.pyplot as plt 
from sklearn.linear_model import Ridge
import numpy as np
import pymc
from tsfresh.feature_extraction import extract_features, MinimalFCParameters, EfficientFCParameters
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute


files = pd.read_excel('/home/velaraptor/Downloads/Raw Data 10yrs (2018).xlsx', header=1)

In [2]:
files = files.fillna(0)
groups = files.groupby('Name')

import tqdm

forecast_df = []
for name, group in tqdm.tqdm(groups):
    if len(group) > 1:
        group.index = group.Year
        df_shift, y = make_forecasting_frame(group["FantPt"], kind=name, max_timeshift=10, rolling_direction=1)
        forecast_df.append(df_shift)

100%|██████████| 2011/2011 [01:18<00:00, 24.68it/s]


In [3]:
f = files[files['Year'] != 2018]

test_f = files[files['Year'] ==2018].Name.unique()

test_ff = files[files['Name'].isin(test_f)]

In [4]:
extracted_features = extract_features(f[['Year', 'Name', 'PPG', 'FantPt']], column_id="Name", column_sort="Year", default_fc_parameters=EfficientFCParameters())

Feature Extraction: 100%|██████████| 20/20 [01:19<00:00,  2.12s/it]


In [5]:
test_features = extract_features(test_ff[['Year', 'Name', 'PPG', 'FantPt']], column_id="Name", column_sort="Year", default_fc_parameters=EfficientFCParameters())

Feature Extraction: 100%|██████████| 20/20 [00:28<00:00,  1.41s/it]


In [6]:
ts_features = extracted_features.reset_index()
ts_features_test = test_features.reset_index()

In [7]:
ts_features = ts_features.dropna(axis=1)
ts_features_test = ts_features_test.dropna(axis=1)

ts_features_test = ts_features_test[ts_features.columns]

idx = files.groupby(['Name'])['Year'].transform(max) == files['Year']
y = files[idx][['Name', 'FantPt', 'Age']]
ts_features = ts_features.merge(y, left_on='id',right_on='Name')

ts_features_test = ts_features_test.merge(y, left_on='id',right_on='Name')


In [ ]:
columns = np.append(ts_features.columns[2:len(ts_features.columns)-3].values, 'Age')
ada = AdaBoostRegressor(base_estimator=GradientBoostingRegressor(loss='quantile', max_depth=10, n_estimators=40, criterion='mae', alpha=0.985), n_estimators=25, loss='exponential')
ada.fit(ts_features[columns], ts_features['FantPt'])

In [ ]:
fig, ax = plt.subplots(figsize=(30,10))
plt.scatter(ada.predict(ts_features[columns]), ts_features['FantPt'])
plt.show()

In [ ]:
pd.DataFrame([ada.predict(ts_features[columns]), ts_features['FantPt']]).T

In [ ]:
from sklearn.metrics import median_absolute_error
median_absolute_error(ts_features['FantPt'], ada.predict(ts_features[columns]))

In [ ]:
final_predictions = pd.DataFrame({'Name': ts_features_test.id, 'prediction': ada.predict(ts_features_test[columns])})



In [ ]:
ts_features_test[columns]

In [ ]:
final_predictions = final_predictions.merge(files[['Name', 'FantPos']].drop_duplicates(), on='Name', how='right').drop_duplicates('Name')

In [ ]:
final_predictions.max()

In [ ]:
idx_1 = final_predictions.groupby(['FantPos'])['prediction'].transform(max) == final_predictions['prediction']

final_predictions[idx_1]